
# Decentralised-Relational-Data-Model-for-Reliability-Studies-of-Medium-Voltage-Cables

One opportunity for DSOs to standardise data collection in the context of reliability prediction of MV cables is to
transform the described mini-world into an entity-relationship data model (ERDM). This not only ensures that all data
adheres to specified formats and relationships but also fulfils a pre-requisite for decentralised data storage. The reason for selecting a relational database over other database designs is that the relational database offersm a structured approach to data modelling and can easily be implemented. Moreover, most of the databases currently in operation are relational, so it is well known, standardised and have a huge user base. Following an ERDM thus facilitates the integration of the proposed data-schema into real-world applications and strengthens its relevance for DSOs and other
Stakeholders.


In [1]:
#Connect to postgresql database and create a table
from sqlalchemy import create_engine, MetaData
from sqlalchemy.orm import declarative_base
from sqlalchemy import Column, Integer, String, Boolean, Date, ForeignKey, Float
from sqlalchemy.exc import ProgrammingError
from sqlalchemy.ext.hybrid import hybrid_property
from sqlalchemy.sql import select, func
from sqlalchemy.orm import sessionmaker, relationship
from geoalchemy2 import Geometry
from sqlalchemy.sql import text

# #TODO: Replace the following with your own details
db_username = ''
db_password = ''
db_host = ''  
db_port = ''
db_name = ''

# Create the database engine
engine = create_engine(f'postgresql+psycopg2://{db_username}:{db_password}@{db_host}:{db_port}/{db_name}')

In [2]:
# #TODO: The name of the database you want to drop and create
DATABASE_NAME = ""

In [4]:
# Connect to the PostgreSQL server
with engine.connect() as conn:
    conn.execution_options(isolation_level="AUTOCOMMIT")

    # Drop the database if it exists
    try:
        conn.execute(text(f"DROP DATABASE IF EXISTS {DATABASE_NAME}"))
        print(f"Database {DATABASE_NAME} dropped.")
    except ProgrammingError as e:
        print(f"Error dropping database {DATABASE_NAME}: {e}")

    # Create the database
    try:
        conn.execute(text(f"CREATE DATABASE {DATABASE_NAME}"))
        print(f"Database {DATABASE_NAME} created.")
    except ProgrammingError as e:
        print(f"Error creating database {DATABASE_NAME}: {e}")


Database MyDB dropped.
Database MyDB created.


In [5]:
with engine.connect() as conn:
    conn.execution_options(isolation_level="AUTOCOMMIT")
    
    # Enable PostGIS extension
    try:
        conn.execute(text("CREATE EXTENSION IF NOT EXISTS postgis"))
        print(f"PostGIS extension enabled for {DATABASE_NAME}.")
    except ProgrammingError as e:
        print(f"Error enabling PostGIS extension for {DATABASE_NAME}: {e}")

PostGIS extension enabled for MyDB.


In [6]:
#NOTE: THE CODE deletes all tables from the database except the required tables for PostGIS

# Create a MetaData instance
metadata = MetaData()

# Reflect the current database tables
metadata.reflect(bind=engine)

# List of required table names for PostGIS
required_tables = ['spatial_ref_sys']

# Drop all tables except the required tables
for table in reversed(metadata.sorted_tables):
    if table.name not in required_tables:
        table.drop(bind=engine)

print("Only the required tables for PostGIS are kept.")


Only the required tables for PostGIS are kept.


In [7]:
# Define your tables using the declarative base
Base = declarative_base()

In [8]:

class DSO(Base):
    __tablename__ = 'dso'
    id = Column(Integer, primary_key=True)
    name = Column(String)
    cvr = Column(Integer)
    geometry = Column(Geometry('GEOMETRY', srid=25832))

In [9]:
# Define Database Tables for Component data, substations, cables, etc. 

class MediumVoltageCableSystem(Base):
    __tablename__ = 'medium_voltage_cable_system'
    id = Column(Integer, primary_key=True)
    dso_id = Column(Integer, ForeignKey('dso.id'))
    station_from_id = Column(Integer, ForeignKey('substation.id'))
    station_to_id = Column(Integer, ForeignKey('substation.id'))
    operating_voltage = Column(Integer)
    #average power in kW
    average_loading = Column(Float)
    #max power in kW
    max_loading = Column(Float)
    time_of_max_loading = Column(Date)
    geometry = Column(Geometry('LINESTRING', srid=25832))

    station_from = relationship("Substation", back_populates="cables_from", foreign_keys=[station_from_id])
    station_to = relationship("Substation", back_populates="cables_to", foreign_keys=[station_to_id])

    # Define the relationship to MediumVoltageCableSubsections
    subsections = relationship("MediumVoltageCableSubsection", back_populates="medium_voltage_cable_system")

    @hybrid_property
    def number_of_subsections(self):
        # This part will be used when accessing the attribute directly from an instance of the model
        return len(self.subsections)
    
    @number_of_subsections.expression
    def number_of_subsections(cls):
        # This part will be used when querying the database
        return select([func.count(MediumVoltageCableSubsection.id)]).where(MediumVoltageCableSubsection.cable_system_id == cls.id)
    
    #calculate the length as a hybrid property
    @hybrid_property
    def length_km(self):
        # This part will be used when accessing the attribute directly from an instance of the model
        return sum(subsection.length_km for subsection in self.subsections)
    
    @length_km.expression
    def length_km(cls):
        # This part will be used when querying the database
        return select([func.sum(MediumVoltageCableSubsection.length_km)]).where(MediumVoltageCableSubsection.cable_system_id == cls.id)

    # calculate the number of joints for the cable system
    @hybrid_property
    def number_of_joints(self):
        # This part will be used when accessing the attribute directly from an instance of the model
        return sum(subsection.number_of_joints for subsection in self.subsections)
    
    @number_of_joints.expression
    def number_of_joints(cls):
        # This part will be used when querying the database
        return select([func.sum(MediumVoltageCableSubsection.number_of_joints)]).where(MediumVoltageCableSubsection.cable_system_id == cls.id)
    
    # calculate the number of repairment sections for the cable system
    @hybrid_property
    def number_of_repairment_sections(self):
        # This part will be used when accessing the attribute directly from an instance of the model
        return sum(subsection.repairment_section for subsection in self.subsections)
    

class Substation(Base):
    __tablename__ = 'substation'
    id = Column(Integer, primary_key=True)
    dso_id = Column(Integer, ForeignKey('dso.id'))
    number_of_consumers = Column(Integer)
    installation_date = Column(Date)
    geometry = Column(Geometry('POINT', srid=25832))
    
    cables_from = relationship("MediumVoltageCableSystem", back_populates="station_from", foreign_keys="[MediumVoltageCableSystems.station_from_id]")
    cables_to = relationship("MediumVoltageCableSystem", back_populates="station_to", foreign_keys="[MediumVoltageCableSystems.station_to_id]")

class MainSubstation(Substation):
    __tablename__ = 'main_substation'
    id = Column(Integer, ForeignKey('substation.id'), primary_key=True)
    name = Column(String(255))
    children_substations = relationship("SecondarySubstation", back_populates="parent_station")

class SecondarySubstation(Substation):
    __tablename__ = 'secondary_substation'
    id = Column(Integer, ForeignKey('substation.id'), primary_key=True)
    parent_station_id = Column(Integer, ForeignKey('main_substation.id'))
    parent_station = relationship("MainSubstation", back_populates="children_substations")

class MediumVoltageCableSubsection(Base):
    __tablename__ = 'medium_voltage_cable_subsection'
    id = Column(Integer, primary_key=True)
    cable_system_id = Column(Integer, ForeignKey('medium_voltage_cable_system.id'))
    #sub_sektion_number = Column(Integer)
    number_of_conductors_primary = Column(Integer)
    conductor_size_primary_mm = Column(Float)  
    conductor_material = Column(String(255))
    insulation = Column(String(255))
    conductor_type = Column(String(255))  
    manufacturer = Column(String(255))
    in_service_date = Column(Date)  
    length_km = Column(Float)  
    coordinates = Column(Geometry('LineString'))
    repairment_section = Column(Boolean)
    out_of_service = Column(Boolean)
    connected_cable_subsection_first = Column(Integer, ForeignKey('medium_voltage_cable_subsection.id'))
    connected_cable_subsection_second = Column(Integer, ForeignKey('medium_voltage_cable_subsection.id'))
    
    medium_voltage_cable = relationship("MediumVoltageCableSystem", back_populates="subsections")
    joins_as_first = relationship("CableJoints", back_populates="first_cable_section", foreign_keys="[CableJoints.first_cable_section]")
    joins_as_second = relationship("CableJoints", back_populates="second_cable_section", foreign_keys="[CableJoints.second_cable_section]")

    @property 
    def number_of_joints(self):
        return len(self.joins_as_first) + len(self.joins_as_second)
    
class CableJoints(Base):
    __tablename__ = 'cable_joints'
    id = Column(Integer, primary_key=True)
    first_cable_section = Column(Integer, ForeignKey('medium_voltage_cable_subsection.id'))
    second_cable_section = Column(Integer, ForeignKey('medium_voltage_cable_subsection.id'))
    joint_type = Column(String(255))
    coordinates = Column(Geometry('POINT'))



In [10]:
class CableEvents(Base):
    __tablename__ = 'cable_events'
    id = Column(Integer, primary_key=True)
    affected_medium_voltage_cable_subsection_id = Column(Integer, ForeignKey('medium_voltage_cable_subsection.id'))
    date = Column(Date)
    #Add more attributes


# Define Database tables for the Failures
class CableFailures(CableEvents):
    __tablename__ = 'cable_failures'

    id = Column(Integer, ForeignKey('cable_events.id'), primary_key=True)
    failure_type = Column(String(255))
    failure_cause = Column(String(255))
    failure_location = Column(Geometry('POINT', srid=25832))
    #add weather conditions 
    #TODO: Add weather attributes during the failure here


class CableRepairs(CableEvents):
    __tablename__ = 'cable_repairs'
    id = Column(Integer, ForeignKey('cable_events.id'), primary_key=True)
    failures_id = Column(Integer, ForeignKey('cable_failures.id'), primary_key=True)
    repairment_cable_section = Column(Integer, ForeignKey('medium_voltage_cable_subsection.id'), primary_key=True)



class ExternalEvents(Base):
    __tablename__ = 'external_events'
    id = Column(Integer, primary_key=True)
    start_date = Column(Date)
    end_date = Column(Date)
    external_event_type = Column(String)
    #Calculate derived attributes e.g. duration
    @hybrid_property
    def duration(self):
        return self.End - self.Start
    
    @duration.expression
    def duration(cls):
        return cls.End - cls.Start  


class DiggingActivities(ExternalEvents):
    __tablename__ = 'digging_activities'
    id = Column(Integer, ForeignKey('external_events.id'), primary_key=True)
    #diggingInquiryNumber = Column(Integer)
    utilityType = Column(String)
    utilityTypeOther = Column(String)
    diggingType = Column(String)
    diggingTypeOther = Column(String)
    geometry = Column(Geometry(geometry_type='GEOMETRY', srid=25832))
    cable_details_requested = Column(Boolean)
 

class Lightnings(ExternalEvents):
    __tablename__ = 'lightning'
    id = Column(Integer, ForeignKey('external_events.id'), primary_key=True)
    Impact_Time = Column(Date)
    coordinates = Column(Geometry('POINT'))
    #TODO: Add features

class Heatwaves(ExternalEvents):
    __tablename__ = 'heatwaves'
    id = Column(Integer, ForeignKey('external_events.id'), primary_key=True)
    Max_Temperature = Column(Float)
    Min_Temperature = Column(Float)
    Average_Temperature = Column(Float) 
    coordinates = Column(Geometry('MultiPolygon'))

class Coldwaves(ExternalEvents):
    __tablename__ = 'coldwaves'
    id = Column(Integer, ForeignKey('external_events.id'), primary_key=True)
    Max_Temperature = Column(Float)
    Min_Temperature = Column(Float)
    Average_Temperature = Column(Float)


class Floods(ExternalEvents):
    __tablename__ = 'floods'
    id = Column(Integer,ForeignKey('external_events.id'), primary_key=True)
    #TODO: Check available data to add features here
    coordinates = Column(Geometry('MultiPolygon'))


# Create an entity to track the events on the cables over time 
class CableExternalEventImpacts(Base):
    __tablename__ = 'cable_external_event_impacts'
    id = Column(Integer, primary_key=True)
    medium_voltage_cable_subsection_id = Column(Integer, ForeignKey('medium_voltage_cable_subsection.id'))
    event_id = Column(Integer, ForeignKey('external_events.id'))
    

In [12]:
# Supertype table for location based drivers
class LocationBasedDrivers(Base):
    __tablename__ = 'location_based_drivers'
    id = Column(Integer, primary_key=True)
    driver_type = Column(String(255))

#Create entities for location based drivers 
class Roads(LocationBasedDrivers):
    __tablename__ = 'roads'
    id = Column(Integer,ForeignKey('location_based_drivers.id'), primary_key=True)
    road_type = Column(String(255))
    coordinates = Column(Geometry('LINESTRING', srid=4326))

class Rails(LocationBasedDrivers):
    __tablename__ = 'rails'
    id = Column(Integer, ForeignKey('location_based_drivers.id'), primary_key=True)
    rail_type = Column(String(255))
    coordinates = Column(Geometry('LINESTRING', srid=4326))

class WaterBodies(LocationBasedDrivers):
    __tablename__ = 'water_bodies'
    id = Column(Integer, ForeignKey('location_based_drivers.id'), primary_key=True)
    water_body_type = Column(String(255))
    coordinates = Column(Geometry('MULTIPOLYGON', srid=4326))

class SoilType(LocationBasedDrivers):
    __tablename__ = 'soil_type'
    id = Column(Integer, ForeignKey('location_based_drivers.id'), primary_key=True)
    soil_type = Column(String(255))
    coordinates = Column(Geometry('MULTIPOLYGON', srid=4326))

#Create a weather conditions entity
class EnvironmentalObservation(LocationBasedDrivers): 
    __tablename__ = 'environmental_observations'
    id = Column(Integer, ForeignKey('location_based_drivers.id'), primary_key=True)
    Average_temperature = Column(Float)
    Average_wind_speed = Column(Float)
    Average_humidity = Column(Float)
    Average_precipitation = Column(Float)
    Max_temperature = Column(Float)
    Min_temperature = Column(Float)
    Max_wind_speed = Column(Float)
    Max_humidity = Column(Float)
    Max_precipitation = Column(Float)
    Time_range_for_calculation_start = Column(Date)
    Time_range_for_calculation_end = Column(Date)
    #location can be added through station or grid values 
    #station_coordinates = Column(Geometry('POINT', srid=4326))
    grid_coordinates = Column(Geometry('MULTIPOLYGON', srid=4326))

class CablePlacementConditions(Base):
    __tablename__ = 'cable_placement_conditions'
    id = Column(Integer, primary_key=True)
    medium_voltage_cable_subsection_id = Column(Integer, ForeignKey('medium_voltage_cable_subsection.id'))
    # Add a foreign key that can originate from the location based drivers
    # Foreign keys should be provided as a list of strings
    location_driver_id = Column(Integer, ForeignKey('location_based_drivers.id'))
    # Add more attributes e.g. spatial overlap percentage? 

In [13]:
# Create all tables stored in this metadata
Base.metadata.create_all(engine)
print("Tables created successfully.")

Tables created successfully.
